### Initial Setup and Data Loading

The notebook begins by importing the core libraries used throughout the project. Each has a distinct role in the workflow:

Pandas serves as the main workhorse for cleaning and transforming the dataset. NumPy supports numerical operations that come up during preprocessing, while Matplotlib and Seaborn are reserved for visual checks that help validate assumptions about the data. The imports here already reflect a lightweight BI-oriented pipeline rather than a heavyweight ML environment.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv(r"C:\Users\SHANIA\Downloads\E-commerce-Data-Insights-Dashboard\data\ecommerce_sales_raw.csv")


Viewing the first few rows is an early check. This confirms that the file path is correct, the delimiter is recognized, and no immediate structural issues appear upon load. Since this is a synthetic dataset meant to behave like transactional E-commerce data, the preview lets us validate that the columns align with expectations such as order_id, customer_id, pricing fields, dates, and categorical descriptors.

After confirming that the file loads cleanly, the next few calls inspect the basic structure:

In [ ]:
df.head()

In [ ]:
df.shape

The dataset contains 34,500 rows and 17 columns, a manageable size suitable for BI dashboards and light predictive work.

In [ ]:
df.columns

The field names are clean and already analysis-ready, removing the need for renaming or manual parsing.

In [ ]:
df.dtypes

Several key fields are initially typed as object strings, including order_date, which will need explicit conversion later so time-based analysis works correctly.

### Missing Values, Duplicates, and Initial Data Quality Checks

This section validates the quality of the dataset before performing deeper preprocessing. Since the file originates from a Kaggle project that simulates E-commerce operations, it is expected to be cleaner than real transactional data, although it still mirrors the structure and statistical behavior of real-world sales systems.

In [ ]:
df.isnull().sum()

Missing Values Report

•  No nulls detected.

•  Expected for synthetic data but simplifies pipeline.

•  No imputation needed for BI or ML baselines.

In [ ]:
df.duplicated().sum() 

No duplicated records were detected. For a transactional dataset with order_id and customer_id, this suggests the generator ensured uniqueness of each transaction. In practical settings, duplicates would normally arise from ingestion errors or repeated API calls, but for this dataset, uniqueness is guaranteed. The check is still essential because it confirms that aggregation, grouping, or potential time-series breakdowns will not be inflated by accidental repeat entries.

In [ ]:
df['order_date'] = pd.to_datetime(df['order_date'], errors='coerce')

The date column arrives as an object type, which prevents temporal analysis. Converting it into a datetime type allows the notebook to later extract month, year, seasonality, or forecast horizons. The errors='coerce' parameter ensures that any incorrectly formatted entries would become NaT instead of breaking the pipeline. Even though no errors appear here, using coercion is a defensive habit typical in real business pipelines.

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols].describe()

Descriptive statistics help verify whether values fall within plausible ranges. Several observations emerge:

• Price values range from 1 to around 2,900, which is plausible for a mixed-category E-commerce platform.

• Delivery times range from 3 to 13 days, behaving like typical 3-7 day windows with a few slower deliveries.

• Profit margins have negative values, which is meaningful because it can imply discounted or loss-leader transactions rather than errors.

• Customer ages range from 18 to 69, a realistic bracket for online shoppers.



In [ ]:
df[numeric_cols].hist(bins=30, edgecolor='black', figsize=(15,10))
dpi = 400

Plotting histograms helps detect skewness, heavy tails, or unnatural spikes. Many features show right skew (price, total amount, profit margin). This is typical for E-commerce, where most orders are low-value and high-ticket items form the long tail.

Even though the dataset is clean, these distributions help confirm that future predictive work should consider normalization or log-scaling for certain models.

In [ ]:
outlier_counts = {}
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[col] < (Q1 - 1.5*IQR)) | (df[col] > (Q3 + 1.5*IQR))]
    outlier_counts[col] = outliers.shape[0]

pd.Series(outlier_counts, name='Potential Outliers')

Applying the IQR rule shows a notable volume of statistically flagged values.  These figures don’t automatically imply data issues. In many E-commerce contexts, wide value ranges can emerge from several factors such as occasional bulk purchases, high-ticket items, sporadic promotions, or returns that affect computed margins. Since the dataset is a simulation of real activity, these patterns may simply reflect diverse customer behaviors rather than anomalies.

Only the outlier counts were computed. No filtering was applied because removing values without a clear business rationale risks distorting spending or operational patterns. At this point, the results are treated as indicators of distribution shape rather than errors. This keeps the dataset aligned with the project’s goal of generating realistic Tableau insights and light predictive exploration.

In [ ]:
categorical_cols = df.select_dtypes(include='object').columns
df[categorical_cols].nunique() 

The cardinality profile looks appropriate:

• Thousands of unique customers and products

• Only a handful of payment methods and regions

• Two values for returned, three for gender

This mix aligns with typical retail data and confirms the dataset is structurally suited for segmentation, cohort analysis, and feature engineering later on.

### Feature Engineering 

In [ ]:
# --- 1. Copy original df to avoid contaminating raw version ---
df_fe = df.copy()

# --- 2. Basic feature engineering ---
df_fe['delivery_speed'] = df_fe['delivery_time_days']

df_fe['discount_bucket'] = pd.cut(
    df_fe['discount'],
    bins=[-0.01, 0.01, 0.10, 0.30],
    labels=['No Discount', 'Low Discount', 'High Discount']
)

df_fe['total_cost_ratio'] = df_fe['shipping_cost'] / df_fe['total_amount']

df_fe['order_value_segment'] = pd.cut(
    df_fe['total_amount'],
    bins=[0, 50, 200, 500, df_fe['total_amount'].max()],
    labels=['Low Value', 'Mid Value', 'High Value', 'Very High']
)

df_fe['returned_flag'] = df_fe['returned'].map({'Yes': 1, 'No': 0})

# --- 3. Advanced / signal-based feature engineering (time-aware) ---
df_fe = df_fe.sort_values(by='order_date')

# Customer-level cumulative features
df_fe['past_orders'] = df_fe.groupby('customer_id').cumcount()
df_fe['past_returns'] = (
    df_fe.groupby('customer_id')['returned_flag']
    .cumsum()
    .shift(fill_value=0)
)

# Product-level cumulative return rate
df_fe['product_returns_cum'] = (
    df_fe.groupby('product_id')['returned_flag']
    .cumsum()
    .shift(fill_value=0)
)
df_fe['product_orders_cum'] = df_fe.groupby('product_id').cumcount()
df_fe['product_return_rate'] = (
    df_fe['product_returns_cum'] / df_fe['product_orders_cum'].replace(0, np.nan)
)

# --- 3b. Handle NaNs ---
df_fe['product_return_rate'] = df_fe['product_return_rate'].fillna(0)
df_fe['past_orders'] = df_fe['past_orders'].fillna(0).astype(int)
df_fe['past_returns'] = df_fe['past_returns'].fillna(0).astype(int)

# --- 4. Extra refined features ---
# Recency: days since last order per customer
df_fe['days_since_last_order'] = (
    df_fe.groupby('customer_id')['order_date']
    .diff()
    .dt.days
    .fillna(9999)  # large value for first order
)

# Discount × category interaction (categorical combo for modeling)
df_fe['discount_category'] = df_fe['discount'].round(2).astype(str) + "_" + df_fe['category']

# Price per unit
df_fe['price_per_unit'] = df_fe['price'] / df_fe['quantity']

# Temporal features
df_fe['order_month'] = df_fe['order_date'].dt.month
df_fe['order_weekday'] = df_fe['order_date'].dt.weekday

print("Shape after refined feature engineering:", df_fe.shape)
print("Sample columns:", df_fe.columns.tolist())



### **Feature Engineering Overview**

The dataset was expanded from 17 to 27 columns, preserving all 34,500 orders and creating **interpretable features that support both dashboards and predictive modeling**. Key engineered features include:

* **Delivery speed (`delivery_speed`)** captures order transit time, allowing analysis of whether longer deliveries influence returns.
* **Discount bucket (`discount_bucket`)** groups orders into ‘No’, ‘Low’, and ‘High’ discount tiers, helping assess how promotions affect return likelihood.
* **Total cost ratio (`total_cost_ratio`)** measures shipping cost relative to order value, highlighting operational stress points that may drive returns.
* **Order value segment (`order_value_segment`)** categorizes orders by purchase size, enabling risk and profitability segmentation.
* **Return flag (`returned_flag`)** converts returns into a 0/1 target for ML while supporting summary metrics in dashboards.
* **Customer-level features (`past_orders`, `past_returns`)** provide historical context, identifying patterns in buyer behavior.
* **Product-level feature (`product_return_rate`)** highlights products prone to returns, guiding inventory and operational decisions.
* **Temporal features (`order_month`, `order_weekday`)** reveal seasonality and weekday effects on sales and returns.

**Impact & questions addressed:** These features allow the business to ask *Which orders or customers are more likely to return products? How do shipping, discounting, or order value affect returns? Which products or periods generate higher risk?* They also provide **interpretable signals for ML models**, enabling accurate prediction of returns while supporting actionable insights in dashboards.

###  ML Pipeline: Predicting Returns from Order Features

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, auc
from sklearn.impute import SimpleImputer
import numpy as np
import matplotlib.pyplot as plt

In [ ]:


# --- 1. Define features ---
numeric_features = [
    'delivery_speed', 'total_cost_ratio', 'past_orders', 'past_returns',
    'product_return_rate', 'customer_age', 'days_since_last_order', 'price_per_unit'
]
categorical_features = [
    'discount_bucket', 'order_value_segment', 'payment_method', 'region',
    'customer_gender', 'discount_category'
]

X = df_fe[numeric_features + categorical_features]
y = df_fe['returned_flag']

# --- 2. Train/test split (stratified) ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# --- 3. Preprocessing ---
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
            ('scaler', StandardScaler())
        ]), numeric_features),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features)
    ]
)

# --- 4. LightGBM Classifier ---
clf_lgbm = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', lgb.LGBMClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=-1,
        random_state=42,
        class_weight='balanced',   # handles imbalance
        scale_pos_weight=len(y_train[y_train==0]) / len(y_train[y_train==1])  # ratio of negatives to positives
    ))
])

clf_lgbm.fit(X_train, y_train)
y_proba_lgbm = clf_lgbm.predict_proba(X_test)[:, 1]

# --- 5. Threshold tuning via PR curve ---
precision, recall, thresholds = precision_recall_curve(y_test, y_proba_lgbm)
f1_scores = (2 * precision * recall) / (precision + recall + 1e-9)
best_idx = np.argmax(f1_scores)
best_threshold = thresholds[max(best_idx - 1, 0)]
y_pred_lgbm = (y_proba_lgbm >= best_threshold).astype(int)

print("LightGBM Confusion Matrix:\n", confusion_matrix(y_test, y_pred_lgbm))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lgbm))
print("\nROC-AUC Score:", roc_auc_score(y_test, y_proba_lgbm))

# --- 6. PR curve visualization ---
plt.figure(figsize=(6,4))
plt.plot(recall, precision, label=f'LGBM PR-AUC = {auc(recall, precision):.2f}')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve (LightGBM)')
plt.legend()
plt.show()

# --- 7. Feature importance ---
feature_names = numeric_features + list(clf_lgbm.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_features))
importances = clf_lgbm.named_steps['classifier'].feature_importances_
indices = np.argsort(importances)[::-1]

print("\nFeature Importance (LightGBM):")
for i in indices[:25]:
    print(f"{feature_names[i]}: {importances[i]:.3f}")


In [ ]:
import os

# --- Add ML predictions to the feature-engineered dataframe ---

# Logistic Regression predictions (probability of return)
df_fe['return_prob_lr'] = clf_lr.predict_proba(X)[:,1]

# Apply the threshold used in training (0.3)
threshold_lr = 0.3
df_fe['return_pred_lr'] = (df_fe['return_prob_lr'] > threshold_lr).astype(int)

# If Random Forest is trained, use rf (or adjust variable name accordingly)
df_fe['return_prob_rf'] = rf.predict_proba(X)[:,1]
df_fe['return_pred_rf'] = (df_fe['return_prob_rf'] >= 0.5).astype(int)

# --- Export to CSV ---
output_folder = r"C:\Users\SHANIA\Downloads\E-commerce-Data-Insights-Dashboard\data"
output_file = "ecommerce_sales_with_predictions.csv"
output_path = os.path.join(output_folder, output_file)
df_fe.to_csv(output_path, index=False)

print(f"Dataset with predictions saved to: {output_path}")
